In [36]:
import os, sys

In [37]:
model_id = 'ContinuousSBM_ContinuousVPSDE_I_BPROJ_bottleneck_32_firstc_10_phi_beta_cosine_betamax_0.5_betamin_0.001'
output_dir ='/mnt/home/dheurtel/ceph/04_inference/sbc/'+model_id
num_samples_per_chain = 500
num_sims_per_gpu = 256
nb_gpus = 12

In [38]:
print(output_dir)

/mnt/home/dheurtel/ceph/04_inference/sbc/ContinuousSBM_ContinuousVPSDE_I_BPROJ_bottleneck_32_firstc_10_phi_beta_cosine_betamax_0.5_betamin_0.001


In [33]:
script_template = \
"""#!/bin/bash
#SBATCH -J sbc_SigmaMNç{{count}}
#SBATCH --gpus=1
#SBATCH --cpus-per-gpu=16
#SBATCH --time=48:00:00
#SBATCH --partition=gpu
#SBATCH --constraint='a100-80gb|h100'
#SBATCH -o jobs/sbc_SigmaMN_{{count}}.log

source ~/.bashrc
source /mnt/home/dheurtel/venv/genv_DL/bin/activate
python sbc_with_MN_and_time.py --model_id {{model_id}} --num_chain={{num_sims_per_gpu}} --num_sample={{num_samples_per_chain}} --save_path={{save_path}}
"""

In [34]:
keys_substitutes = {"{{model_id}}": model_id,
                    "{{num_sims_per_gpu}}": num_sims_per_gpu,
                    "{{num_samples_per_chain}}": num_samples_per_chain,
                    "{{save_path}}": os.path.join(output_dir, "tmp.pt"),
                    "{{count}}": 0}

In [35]:
for gpu_id in range(nb_gpus):
    keys_substitutes["{{count}}"] = gpu_id
    keys_substitutes["{{save_path}}"] = os.path.join(output_dir, f"sbc_sims_SigmaMN_{gpu_id}.pt")
    script = script_template
    for key, value in keys_substitutes.items():
        script = script.replace(key, str(value))
    with open(f'./jobs/sbc_sims_SigmaMN_{gpu_id}.sh', 'w') as f:
        f.write(script)
    os.system(f'sbatch ./jobs/sbc_sims_SigmaMN_{gpu_id}.sh')

Submitted batch job 3117266
Submitted batch job 3117267
Submitted batch job 3117268
Submitted batch job 3117269
Submitted batch job 3117270
Submitted batch job 3117271
Submitted batch job 3117272


Submitted batch job 3117273
Submitted batch job 3117274
Submitted batch job 3117275
Submitted batch job 3117276
Submitted batch job 3117277
